In [146]:
from collections import Counter
import nltk
import random
import bisect

In [50]:
#Read and format data
text = open("berp-training.txt").read()
words = text.split()
wordFreqs = Counter(words)
wordValues = list(wordFreqs.values())

#Replae freq 1 words with <unk>
for i in range(len(words)):
    if wordFreqs[words[i]] == 1:
        words[i] = "<unk>"
wordFreqs = Counter(words)

total = sum(freqs.values())
bigrams = list(nltk.bigrams(words))

#Remove ('</s>', '<s>') bigrams
for pair in bigrams:
    if pair == ('</s>', '<s>'):
        bigrams.remove(pair)
        
bigramFreqs = Counter(bigrams)
        
bigramTotal = len(bigrams)
wordTotal = sum(wordFreqs.values())

In [120]:
def unigramModel(sentence):
    sentence = sentence.split()
    prob = 1
    for i in range(len(sentence)):
        freq = wordFreqs[sentence[i]]
        if freq == 0:#Check for unknown words
            sentence[i] = "<unk>"
        prob = prob * (wordFreqs[sentence[i]]/wordTotal)
    return prob

In [130]:
def bigramModel(sentence):
    prob = 1
    sentenceWords = sentence.split()
    for i in range(len(sentenceWords)):        
        freq = wordFreqs[sentenceWords[i]]
        if freq == 0:#Check for unknown words
            sentenceWords[i] = "<unk>"
    sentenceBigrams = list(nltk.bigrams(sentenceWords))
    for bigram in sentenceBigrams:
        prob = prob * ((bigramFreqs[bigram]+1)/(wordFreqs[bigram[0]]+1)) #add-1 smoothing, bi count+1/prefix uni count+1
    return prob

In [121]:
unigramModel("<s> i want food jay </s>")

1.2304013676520618e-09

In [131]:
bigramModel("<s> i want food jay </s>")

6.705102267922801e-07

In [241]:
f = open('test-phrases.txt', "r")
fu = open('assgn2-unigram-out.txt','a')
fb = open('assgn2-bigram-out.txt','a')
lines = f.readlines()
for line in lines:
    fu.write(str(unigramModel(line))+"\n")
    fb.write(str(bigramModel(line))+"\n")
f.close()
fu.close()
fb.close()

In [147]:
def getSampleBins(wordprobs):
    index = 0; bins = []
    for w in wordprobs:
        index += w
        bins.append(index)
    return bins

def sampleWords(bins):
    rndIndex = random.random()
    return bisect.bisect(bins, rndIndex)

In [230]:
def randBigram(): 
    currWord = "<s>"
    senc = []
    while currWord != "</s>":
        nexts = []
        wordprobs = []
        for i in range(len(bigrams)):#Find every bigram with current word
            if bigrams[i][0] == currWord:
                nexts.append(i)
        for i in range(len(nexts)):#Make list of probabilities for each possible next word
            wordprobs.append(bigramFreqs[bigrams[nexts[i]]]/len(nexts))
            
        bins = getSampleBins(wordprobs)
        currWord = bigrams[nexts[sampleWords(bins)]]#Randomly select next word based on probabilistic weight
        senc.append(currWord)
        currWord = currWord[1]
    return senc

In [231]:
randBigram()

[('<s>', 'uh'),
 ('uh', 'i'),
 ('i', 'have'),
 ('have', 'dinner'),
 ('dinner', '</s>')]

In [225]:
#Create random sentences and write to file
n = 100 #Number of sentences
file = open('assgn2-bigram-rand-corpus.txt','a')
for i in range(n):
    string = ""
    sent = randBigram()
    for i in range(len(sent)):
        string += sent[i][0] + " "
    string += "</s>" + "\n"
    file.write(string) 
file.close() 